In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

In [ ]:
# from dotenv import load_dotenv

# load_dotenv()

True

In [3]:
import pandas as pd

In [4]:
books = pd.read_csv('book_prepared.csv')

In [5]:
#since lanchain cant process df we are creating a text file
books['tagged_description'].to_csv('tagged_description.txt',sep='\n',index=False,header=False)


In [6]:
raw_documents = TextLoader('tagged_description.txt').load()
text_Splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap = 0,separator = "\n")
documents =text_Splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 1000
Created a chunk of size 1214, which is longer than the specified 1000
Created a chunk of size 1088, which is longer than the specified 1000
Created a chunk of size 1189, which is longer than the specified 1000
Created a chunk of size 1267, which is longer than the specified 1000
Created a chunk of size 2010, which is longer than the specified 1000
Created a chunk of size 1225, which is longer than the specified 1000
Created a chunk of size 1184, which is longer than the specified 1000
Created a chunk of size 1214, which is longer than the specified 1000
Created a chunk of size 1191, which is longer than the specified 1000
Created a chunk of size 1057, which is longer than the specified 1000
Created a chunk of size 1270, which is longer than the specified 1000
Created a chunk of size 1635, which is longer than the specified 1000


Created a chunk of size 1132, which is longer than the specified 1000
Created a chunk of size 1325, which is longer than the specified 1000
Created a chunk of size 1119, which is longer than the specified 1000
Created a chunk of size 1195, which is longer than the specified 1000
Created a chunk of size 2012, which is longer than the specified 1000
Created a chunk of size 1286, which is longer than the specified 1000
Created a chunk of size 1231, which is longer than the specified 1000
Created a chunk of size 1242, which is longer than the specified 1000
Created a chunk of size 1126, which is longer than the specified 1000
Created a chunk of size 1073, which is longer than the specified 1000
Created a chunk of size 1318, which is longer than the specified 1000
Created a chunk of size 2834, which is longer than the specified 1000
Created a chunk of size 1141, which is longer than the specified 1000
Created a chunk of size 1004, which is longer than the specified 1000
Created a chunk of s

In [7]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [8]:

# db_books = Chroma.from_documents(documents,embedding=OpenAIEmbeddings())

In [9]:

# pip install ollama

In [10]:
from langchain.embeddings.base import Embeddings
import ollama

class OllamaEmbeddings(Embeddings):
    def __init__(self, model: str = "deepseek-r1"):
        self.model = model

    def embed_documents(self, texts):
        return [self.embed_query(text) for text in texts]

    def embed_query(self, text):
        response = ollama.embeddings(model=self.model, prompt=text)
        return response["embedding"]


In [11]:
ollama_embedding = OllamaEmbeddings(model="deepseek-r1")
db_books = Chroma.from_documents(documents, embedding=ollama_embedding)

In [12]:
query = "A book to teach childern about future aspirations"
docs = db_books.similarity_search(query,k=10)
docs

[Document(id='d8faeaef-13bf-456a-badf-d81c772f0b27', metadata={'source': 'tagged_description.txt'}, page_content='9781565121829 The identity of the Sharpshooter becomes a central theme in a story that introduces such prospective characters as the owner of the local funeral parlor, an avid comic-book reader, a man who shoots refrigerators, and a boy who never grew up'),
 Document(id='c0eb3ef1-9f93-4bee-8919-76ce4bdeadb6', metadata={'source': 'tagged_description.txt'}, page_content="9780156260398 Entries interrupted only by her periodic breakdowns record the daily events and activities, enthusiasms and disappointments, and writing tasks in Virginia Woolf's life and her responses to people, books, and her own work\n9780156260404 Virginia Woolf was fifty-four on January 25, 1936, some three weeks after this final volume of her diary opens. Its last page was written four days before she drowned herself on March 28, 1941. Edited by Anne Olivier Bell, assisted by Andrew McNeillie; Index; maps

I want to recommend the names and details of the books to the user 

In [13]:
books[books['isbn13']==int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
4637,9781565121829,1565121821,The Sharpshooter Blues,Lewis Nordan,Fiction,http://books.google.com/books/content?id=aPuE3...,The identity of the Sharpshooter becomes a cen...,1997.0,4.15,398.0,371.0,The Sharpshooter Blues,9781565121829 The identity of the Sharpshooter...


In [14]:
def retrieve_semantic_recommendations(query:str,top_k:int=10)->pd.DataFrame:
    recs  = db_books.similarity_search(query,k=50)

    books_list = []

    for i in range(len(recs)):
        books_list+= [int(recs[i].page_content.strip('"').split()[0])]

    return books[books['isbn13'].isin(books_list)].head(top_k)

In [15]:
retrieve_semantic_recommendations('Books with inspiring life stories')

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
83,9780029131695,0029131693,Tyranny of the Majority,Lani Guinier,History,http://books.google.com/books/content?id=aRF7X...,At last...the public hearing she was denied......,1995.0,3.82,352.0,39.0,Tyranny of the Majority Funamental Fairness in...,9780029131695 At last...the public hearing she...
529,9780099498636,0099498634,"Victoria Line, Central Line",Maeve Binchy,London (England),http://books.google.com/books/content?id=V7kjF...,A vintage collection of short stories from the...,2006.0,3.59,432.0,928.0,"Victoria Line, Central Line",9780099498636 A vintage collection of short st...
568,9780140139976,0140139974,Sailor Song,Ken Kesey,Fiction,http://books.google.com/books/content?id=-pPSO...,"In Alaska to film a famous children's book, th...",1993.0,3.57,533.0,1956.0,Sailor Song,9780140139976 In Alaska to film a famous child...
580,9780140158762,0140158766,Chickenhawk - Back in the World,Robert Mason,Biography & Autobiography,http://books.google.com/books/content?id=R7cGA...,From the author of the bestselling autobiograp...,1994.0,4.05,400.0,166.0,Chickenhawk - Back in the World Life After Vie...,9780140158762 From the author of the bestselli...
598,9780140237535,0140237534,For Esmé - with Love and Squalor,Jerome David Salinger,Fiction,http://books.google.com/books/content?id=GGprQ...,Indhold: A perfect day for bananafish ; Uncle ...,1994.0,4.19,149.0,2268.0,For Esmé - with Love and Squalor,9780140237535 Indhold: A perfect day for banan...
917,9780156032971,015603297X,Foucault's Pendulum,Umberto Eco,Fiction,http://books.google.com/books/content?id=Ubkvm...,"Three editors, inspired by an extraordinary fa...",2007.0,3.90,623.0,44658.0,Foucault's Pendulum,"9780156032971 Three editors, inspired by an ex..."
925,9780156260398,0156260395,The Diary of Virginia Woolf: 1931-1935,Virginia Woolf,Biography & Autobiography,http://books.google.com/books/content?id=ai7uA...,Entries interrupted only by her periodic break...,1983.0,4.44,420.0,213.0,The Diary of Virginia Woolf: 1931-1935,9780156260398 Entries interrupted only by her ...
928,9780156272506,0156272504,The Early Diary of Anaïs Nin: 1923-1927,Anaïs Nin,Biography & Autobiography,http://books.google.com/books/content?id=dz86m...,A bridge between the early life of Nin and the...,1985.0,4.18,320.0,143.0,The Early Diary of Anaïs Nin: 1923-1927,9780156272506 A bridge between the early life ...
1080,9780241912720,0241912725,Stuart Little,Elwyn Brooks White,"Little, Stuart (Fictitious character)",NaN,The adventures of the debonair mouse Stuart Li...,1946.0,3.89,144.0,109.0,Stuart Little,9780241912720 The adventures of the debonair m...
1306,9780312953966,0312953968,The Edge of Honor,P. T. Deutermann,Fiction,http://books.google.com/books/content?id=8pCVI...,Leaving his home and wife for a tour of duty i...,1995.0,3.96,626.0,162.0,The Edge of Honor,9780312953966 Leaving his home and wife for a ...
